In [5]:
import numpy as np
import xarray as xr
import pandas as pd

# === Step 1: Load CHIRPS daily NetCDF ===
chirps_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\Wflow - WitteveenBos\wflow_JMS\HydroMT\CHIRPS_forcing\stacked_renamed_CHIRPS_rainfall_2007_2021.nc"
ds_daily = xr.open_dataset(chirps_fp)

display(ds_daily)



<xarray.Dataset> Size: 2MB
Dimensions:      (lon: 7, lat: 10, time: 5479)
Coordinates:
  * lon          (lon) float64 56B 35.88 35.93 35.98 36.03 36.08 36.13 36.18
  * lat          (lat) float64 80B -0.225 -0.275 -0.325 ... -0.575 -0.625 -0.675
  * time         (time) datetime64[ns] 44kB 2007-01-01 2007-01-02 ... 2021-12-31
Data variables:
    spatial_ref  int32 4B ...
    precip       (time, lat, lon) float32 2MB ...

In [6]:
rain_var = 'precip'  # Adjust if needed

# === Step 2: Create rainfall hourly profiles ===
def create_convective_pattern():
    """Convective: heavy rain during few afternoon hours."""
    pattern = np.zeros(24)
    start_hour = np.random.randint(12, 15)  # Afternoon peak
    duration = np.random.randint(1, 4)  # 1–3 hours
    rain_perc = np.random.dirichlet(np.ones(duration))  # Random fractions
    pattern[start_hour:start_hour + duration] = rain_perc
    return pattern

def create_stratiform_pattern():
    """Stratiform: light rain over many hours."""
    pattern = np.zeros(24)
    start_hour = np.random.randint(0, 6)  # Early morning start
    duration = np.random.randint(6, 13)  # 6–12 hours
    rain_perc = np.random.dirichlet(np.ones(duration)) * 0.8  # Spread out
    pattern[start_hour:start_hour + duration] = rain_perc
    return pattern

def create_mixed_pattern():
    """Mixed: stratiform light + convective peak."""
    pattern = create_stratiform_pattern()
    conv = create_convective_pattern()
    mixed = pattern + conv
    mixed /= mixed.sum()
    return mixed

# === Step 3: Disaggregation per pixel ===
times_daily = ds_daily['time'].values
lats = ds_daily['lat'].values
lons = ds_daily['lon'].values

new_times = pd.date_range(
    start=str(times_daily[0]),
    end=str(times_daily[-1]) + " 23:00",
    freq="H"
)

hourly_rain_list = []

for t_idx, t_day in enumerate(times_daily):
    daily_rain = ds_daily[rain_var].isel(time=t_idx)

    # Randomly decide rain type
    rain_type = np.random.choice(['convective', 'stratiform', 'mixed'], p=[0.5, 0.3, 0.2])

    if rain_type == 'convective':
        pattern = create_convective_pattern()
    elif rain_type == 'stratiform':
        pattern = create_stratiform_pattern()
    else:
        pattern = create_mixed_pattern()

    # Multiply daily rainfall by pattern
    for h in range(24):
        hourly_rain = daily_rain * pattern[h]
        hourly_rain_list.append(hourly_rain.expand_dims(time=[pd.Timestamp(t_day) + pd.Timedelta(hours=h)]))

# === Step 4: Concatenate all hours ===
ds_hourly = xr.concat(hourly_rain_list, dim="time")

# === Step 5: Save to NetCDF ===
output_fp = "chirps_hourly_physical_disagg.nc"
#ds_hourly = ds_hourly.rename({rain_var: "rainfall"})
ds_hourly.to_netcdf(output_fp)

print(f"✅ Hourly rainfall saved to: {output_fp}")


C:\Users\jmsch\AppData\Local\Temp\ipykernel_28888\4098966422.py:35: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  new_times = pd.date_range(


✅ Hourly rainfall saved to: chirps_hourly_physical_disagg.nc
